# Final Project 1: Population and Agricultural Production Changes In Countries in Response to [Operation Condor]

## Team Malthus: Celeste Basken, Jocelyn  Velazquez, Reily Fairchild, Ziheng Zhang

# Table of Contents
1. [Introduction](#Introduction)
2. [Population Function, Deliverable 1A- Population Statistics](#Population-Function)
3. [Population DataFrame Function, Deliverable 3A - Population DataFrames](#Population-Dataframe)
4. [Regular Population Pyramids, Deliverable 4B - Population Pyramids](#Population-Pyramids)
5. [Animated Population Pyramids](#Results)
6. [Visaulizations](#Visualizations)
7. [Conclusion](#Conclusion)


Unit Tests, Deliverable 2[A] - Unit Tests


Introduction: The data exploration done in this notebook is aimed at exploring the impact and repurcussions done to countries in South America, primarily [Brazil, Argentina, ..] as a result of [].


## Initial Imports

In [9]:
#%pip install wbdata
#import math
#import numpy as np
#import wbdata


## Population Function

### This function returns a query response to the question: 'In [year] how many [people/males/females] aged [low] to [high] were living in [the world/region/country]?'

 Please note, this function was designed to be able to answer **only** the regions of countries targeted in Operation Condor, as that is the main focus region for our team.

In [27]:

import math
#def population(year,sex,age_low, age_high,place):
SOURCE = 40

def age_floor_lessthan_25(age_low, age_high, labels, sex):
    
    for i in np.arange(age_low, age_high+1):
            if sex == 'males':
                #labels[f'SP.POP.AG{str(i).zfill(2)}.MA.IN'] = f'SP.POP.AG{str(i).zfill(2)}.MA.IN'
                labels[f'SP.POP.AG{str(i).zfill(2)}.MA.IN'] = f'Males {str(i).zfill(2)}'
            elif sex == 'females':
                labels[f'SP.POP.AG{str(i).zfill(2)}.FE.IN'] = f'Females {str(i).zfill(2)}'
                #labels[f'SP.POP.AG{str(i).zfill(2)}.FE.IN'] = f'SP.POP.AG{str(i).zfill(2)}.FE.IN'
            else:
                #labels[f'SP.POP.AG{str(i).zfill(2)}.MA.IN'] = f'SP.POP.AG{str(i).zfill(2)}.MA.IN'
                #labels[f'SP.POP.AG{str(i).zfill(2)}.FE.IN'] = f'SP.POP.AG{str(i).zfill(2)}.FE.IN'   
                labels[f'SP.POP.AG{str(i).zfill(2)}.FE.IN'] = f'Females {str(i).zfill(2)}'
                labels[f'SP.POP.AG{str(i).zfill(2)}.MA.IN'] = f'Males {str(i).zfill(2)}'
                
def age_floor_greaterthan_25(age_low, age_high, labels, sex):
        age_floor = math.floor(age_low /5) * 5
        age_ceiling = math.ceil((age_high+1)/5)*5 
        floor_str = str(age_floor).zfill(2)
        num_ranges = (age_ceiling - age_floor) / 5
        for i in np.arange(num_ranges):
            lower_str = str(int(age_floor + 5 * i)).zfill(2)
            higher_str = str(int(age_floor + 5 * i + 4)).zfill(2)

           # lower_str = str(int(age_floor+5*i))
           # higher_str = str(int(age_floor+5*i+4))
           
             #override if age is >=80
            if lower_str >='80':
                lower_str = "80"
                higher_str = 'UP'
            if sex == 'males':
                labels[f'SP.POP.{lower_str}{higher_str}.MA'] = f'Males {lower_str}{higher_str}'
            elif sex == 'females':
                labels[f'SP.POP.{lower_str}{higher_str}.FE'] = f'Females {lower_str}{higher_str}'
            else:
                #labels[f'SP.POP.AG{str(i).zfill(2)}.FE.IN'] = f'SP.POP.AG{str(i).zfill(2)}.FE'
                labels[f'SP.POP.{lower_str}{higher_str}.FE'] = f'Females {lower_str}{higher_str}'
                labels[f'SP.POP.{lower_str}{higher_str}.MA'] = f'Males {lower_str}{higher_str}'


#correct age range, use specific age year values if do not return null:
def range(age_low, age_high, sex): 
    labels = {}
    
    labels_test1 = {}
    age_floor_lessthan_25(0, 24, labels_test1, 'males')
    assert 'SP.POP.AG00.MA.IN' in labels_test1 and 'SP.POP.AG24.MA.IN' in labels_test1, "Failed to retrieve data for ages < 25"

    labels_test2 = {}
    age_floor_greaterthan_25(25, 39, labels_test2, 'females')
    assert 'SP.POP.2529.FE' in labels_test2 and 'SP.POP.3539.FE' in labels_test2, "Failed to retrieve data for ages >= 25"

    #age range [0,25]
    if age_high < 25:
        age_floor_lessthan_25(age_low, age_high, labels, sex)
    #age range [25, +]
    elif age_low >= 25:
        age_floor_greaterthan_25(age_low, age_high, labels, sex)
    #age range [0, 25+]
    else:
        age_floor_lessthan_25(age_low, 24, labels, sex)
        age_floor_greaterthan_25(25, age_high, labels, sex)
        
    return labels

#correct age range, use specific age year values (0,25) do return null:
def age_range_intv(age_low, age_high, sex):
    labels = {}
    
    labels_test2 = {}
    age_floor_greaterthan_25(25, 39, labels_test2, 'females')
    assert 'SP.POP.2529.FE' in labels_test2 and 'SP.POP.3539.FE' in labels_test2, "Failed to retrieve data for ages >= 25"

     
    age_floor_greaterthan_25(age_low, age_high, labels, sex)
   
    return labels



def get_region_df(labels, age_high, sex, place, year):
    df = wbdata.get_dataframe(labels,country = place,parse_dates=True)
    df = df.reset_index()
    df2 = df[df['date'] == f'{year}-01-01']

    #if the age high is >25 and <80 and not a multiple of 5, then take a proportional fraction of the last column
    #example: 27, in range [25, 29] want (27%5 /5)
    #doesn't account for ages 81, 82, 83, etc (difficult to interpolate when so little data on older ages)
    if ((age_high>=25) & (age_high<80) & (age_high%5 != 4)):
        
        population_count = df2.iloc[0, -1]
        display(f'population count: {population_count}')
        proportion = (age_high%5+1) / 5
        df2.iloc[0, -1] = proportion*population_count
        
    return df2
#df = get_df(labels, sex, place, year)
#labels = range(22, 77, 'males')
#df = get_df(labels, 77, 'females', 'COL', 2022)
#df


def population(sex, age_low, age_high, place, year):
    # Error handling for invalid age range
    if age_low > age_high:
        raise ValueError(f"Invalid age range: ({age_low}) < ({age_high}). Please reverse the input age ranges.")

    # Validate `place` is a valid location in wbdata
    ls_countries = wbdata.get_countries()
    country_codes = [country['id'] for country in ls_countries]
    
    if place not in country_codes:
        raise ValueError(f"Invalid country code: {place}. Please enter a valid country. Call wbdata.get_countries to see the valid list of countries.")


    labels = range(age_low, age_high, sex)
     
    #unit test for age edge cases (if highest age lies in an interval)
    tlabels = {'SP.POP.2529.MA': 'Males 25-29'}
    df_test = get_region_df(tlabels, 27, sex, place, year)
    assert df_test.iloc[0, -1] <= df_test.iloc[0, 1], "Population proportion adjustment error!"


    df = get_region_df(labels, age_high, sex, place, year)
    
    # Check if the dataframe has any 'None' values for an age (Like 'WLD')
    if df.iloc[:, 1:].isnull().all().any():
        print('The age range (1, 25) returned null values, using interval ages ranges [0-5] instead')
        labels = age_range_intv(age_low, age_high, sex)
        df = get_region_df(labels, age_high, sex, place, year)
    
    df.drop('date', axis = 1, inplace=True)
    df['total pop in age range'] = df.sum(axis=1)
    total = df.iloc[0]['total pop in age range']
    display(f'In {year}, the number of {sex} between ages {age_low} to {age_high} living in {place} was approximately {total}')
    return df


In [28]:
#ENTER YOUR QUERIES HERE
#df_test = population('males', 0, 100, 'COL', 2000)
#assert df_test.iloc[0]['total pop in age range'] > 1e9 / 2, "Too few males in world population!"

population('males', 1, 24, 'WLD', 2000)
#df_test

'lower string: 25, higher string: 29'

'lower string: 30, higher string: 34'

'lower string: 35, higher string: 39'

'population count: 256111683.0'

The age range (1, 25) returned null values, using interval ages ranges [0-5] instead


...

'lower string: 15, higher string: 19'

'lower string: 20, higher string: 24'

{'SP.POP.0004.MA': 'Males 0004',
 'SP.POP.0509.MA': 'Males 0509',
 'SP.POP.1014.MA': 'Males 1014',
 'SP.POP.1519.MA': 'Males 1519',
 'SP.POP.2024.MA': 'Males 2024'}

'In 2000, the number of males between ages 1 to 24 living in WLD was approximately 1524729165.0'

,Males 0004,Males 0509,Males 1014,Males 1519,Males 2024,total pop in age range
23,323242397.0,319575048.0,322416810.0,295149165.0,264345745.0,1.524729e+09


## Population Dataframe
This function returns a pandas DataFrame indexed by Region or Country and Year, with columns giving counts of people in different age-sex groups. For the means of Team Malthus' project focus, this function was designed to produce specifics for **only** regions/countries we are focussing on: the countries directly targeted impacted by Operation Condor.


In [29]:
condor_countries = {'ARG': 'Argentina', 'BOL': 'Bolivia', 'BRA': 'Brazil', 'CHL': 'Chile', 'ECU': 'Ecuador', 'CLA':'Latin America and the Caribbean (IFC classification)', 'WLD' :"World"}
#labels = range(22, 77, 'females')
#df = get_df(labels, 77, 'females', 'COL', 2022)
#df


def get_region_df(age_low, age_high, sex, place, year):
    labels = range(age_low, age_high, sex)
    df = wbdata.get_dataframe(labels,country = place,parse_dates=True)
    df = df.reset_index()
    df2 = df[df['date'] == f'{year}-01-01']
    df2['date'] = year
    #if the age high is >25 and <80 and not a multiple of 5, then take a proportional fraction of the last column
    #example: 27, in range [25, 29] want (27%5 /5)
    if ((age_high>=25) & (age_high<80) & (age_high%5 != 4)):
        population_count = df2.iloc[0, -1]
        display(f'population count: {population_count}')
        proportion = (age_high%5+1) / 5
        df2.iloc[0, -1] = proportion*population_count
        
        
    return df2

get_region_df(22, 77, 'females',condor_countries, 2021)

'lower string: 25, higher string: 29'

'lower string: 30, higher string: 34'

'lower string: 35, higher string: 39'

'lower string: 25, higher string: 29'

'lower string: 30, higher string: 34'

...

'lower string: 70, higher string: 74'

'lower string: 75, higher string: 79'

/tmp/ipykernel_2266/1665431826.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date'] = year


'population count: 602985.0'

,country,date,Females 22,Females 23,Females 24,Females 2529,Females 3034,Females 3539,Females 4044,Females 4549,Females 5054,Females 5559,Females 6064,Females 6569,Females 7074,Females 7579
2,Argentina,2021,340990.0,339364.5,341034.0,1727965.0,1651617.0,1583375.0,1548702.0,1372580.0,1183031.0,1095091.0,1017473.0,914976.0,777595.0,361791.0
66,Bolivia,2021,109809.5,108289.5,107373.5,518258.0,469786.0,400557.0,349190.0,297321.0,251647.0,210244.0,169351.0,128503.0,96082.0,68510.0
130,Brazil,2021,1649095.0,1655825.0,1646923.0,8105921.0,8299446.0,8539224.0,8027939.0,7135873.0,6449326.0,6076687.0,5236762.0,4186711.0,3107121.0,2055915.0
194,Chile,2021,140459.5,145160.0,149702.0,792156.0,795033.0,725313.0,675065.0,649398.0,629116.0,586850.0,520887.0,436213.0,342926.0,236452.0
258,Ecuador,2021,155491.5,155337.5,155166.0,755239.0,701586.0,648686.0,574585.0,505627.0,442048.0,377253.0,313465.0,256287.0,198079.0,137218.0
322,World,2021,NaN,NaN,NaN,290638670.0,296697604.0,274546997.0,246000086.0,234930861.0,223923211.0,202156613.0,165292891.0,144946891.0,108698089.0,71377606.0


## Regular Population Pyramids